# Agent: Pydantic

간단하고 실용적인 예제를 통해 OpenAI API와 Pydantic을 함께 사용하는 방법을 배운다.

In [2]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

# API 키 확인 (선택사항)
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List
import json

MODEL = "gpt-4o-mini"

client = OpenAI()

## 예제 1: Pydantic 기본 - 데이터 검증

Pydantic은 데이터 유효성 검사 및 설정 관리를 위한 라이브러리입니다. 
클래스를 정의하여 데이터의 형태(스키마)를 명확하게 지정하고, 입력된 데이터가 해당 형태를 따르는지 자동으로 검증해줍니다.

In [4]:
class User(BaseModel):
    """사용자 정보를 정의하는 Pydantic 모델입니다."""
    name: str
    age: int
    email: str

# 사용 예시
# 정의된 타입(str, int)에 맞지 않는 데이터가 들어오면 오류가 발생합니다.
user = User(name="김철수", age=25, email="kim@example.com")

print("예제 1: Pydantic 기본")
# model_dump()는 객체를 딕셔너리로 변환합니다.
print(user.model_dump())

# model_dump_json()은 객체를 JSON 문자열로 변환합니다.
print(user.model_dump_json())

예제 1: Pydantic 기본
{'name': '김철수', 'age': 25, 'email': 'kim@example.com'}
{"name":"김철수","age":25,"email":"kim@example.com"}


## 예제 2: OpenAI Function Calling에 Pydantic 사용

OpenAI의 Function Calling 기능을 사용할 때, 함수의 인자(parameter)를 JSON 스키마 형태로 정의해야 합니다.
Pydantic 모델의 `.model_json_schema()` 메서드를 사용하면 이 스키마를 매우 간편하게 생성할 수 있습니다.

In [5]:
class WeatherInput(BaseModel):
    """날씨 조회 함수의 입력을 정의하는 Pydantic 모델입니다."""
    city: str = Field(description="도시 이름")
    unit: str = Field(default="celsius", description="온도 단위 (celsius, fahrenheit)")

def get_weather(city: str, unit: str = "celsius"):
    """실제 날씨 정보를 가져오는 함수(가상)입니다."""
    return {
        "city": city,
        "temperature": 22,
        "unit": unit,
        "condition": "맑음"
    }

# Pydantic 모델을 OpenAI Function Calling 스키마로 변환합니다.
weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "특정 도시의 현재 날씨 정보를 가져옵니다",
        "parameters": WeatherInput.model_json_schema() # 이 부분이 핵심입니다.
    }
}

print("예제 2: Function Tool 정의")
print(json.dumps(weather_tool, indent=2, ensure_ascii=False))

예제 2: Function Tool 정의
{
  "type": "function",
  "function": {
    "name": "get_weather",
    "description": "특정 도시의 현재 날씨 정보를 가져옵니다",
    "parameters": {
      "description": "날씨 조회 함수의 입력을 정의하는 Pydantic 모델입니다.",
      "properties": {
        "city": {
          "description": "도시 이름",
          "title": "City",
          "type": "string"
        },
        "unit": {
          "default": "celsius",
          "description": "온도 단위 (celsius, fahrenheit)",
          "title": "Unit",
          "type": "string"
        }
      },
      "required": [
        "city"
      ],
      "title": "WeatherInput",
      "type": "object"
    }
  }
}


## 예제 3: OpenAI API로 Function Calling 실행

앞서 정의한 `weather_tool`을 사용하여 실제로 Function Calling을 실행하는 과정입니다.
사용자의 질문을 보고 AI가 `get_weather` 함수를 호출해야겠다고 판단하면, 필요한 인자(`city`)와 함께 함수 호출을 요청합니다.
그러면 우리는 그 요청에 따라 실제 함수를 실행하고, 결과를 다시 AI에게 알려주어 최종 답변을 생성하게 합니다.

In [7]:
def simple_function_call(user_query: str):
    """간단한 Function Calling을 시연하는 함수입니다."""
    
    messages = [{"role": "user", "content": user_query}]
    
    # 1단계: AI에게 사용자 메시지와 함께 사용 가능한 도구(함수) 목록을 전달합니다.
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=[weather_tool]
    )
    
    message = response.choices[0].message
    
    # 2단계: AI가 함수 호출을 결정했는지 확인합니다.
    if message.tool_calls:
        tool_call = message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        print(f"🔧 AI가 선택한 함수: {function_name}")
        print(f"   인자: {arguments}")
        
        # 실제 함수를 실행합니다.
        result = get_weather(**arguments)
        print(f"   결과: {result}")
        
        # 3단계: 함수 실행 결과를 AI에게 다시 전달하여 최종 답변을 요청합니다.
        messages.append(message) # AI의 함수 호출 요청 메시지 추가
        messages.append({        # 실제 함수 실행 결과 메시지 추가
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps(result, ensure_ascii=False)
        })
        
        # 4단계: 최종 응답을 받습니다.
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        
        return final_response.choices[0].message.content
    else:
        # AI가 함수를 호출하지 않고 바로 답변한 경우입니다.
        return message.content

print("예제 3: Function Calling 실행")
result = simple_function_call("서울 날씨 알려줘")
print(f"\n AI 최종 응답: {result}")

예제 3: Function Calling 실행
🔧 AI가 선택한 함수: get_weather
   인자: {'city': '서울'}
   결과: {'city': '서울', 'temperature': 22, 'unit': 'celsius', 'condition': '맑음'}

 AI 최종 응답: 현재 서울의 날씨는 맑고, 기온은 22도입니다. 변동이 있을 수 있으니 참고하세요!


## 예제 4: Structured Output - 원하는 형식으로 응답 받기

Pydantic 모델을 `response_format`으로 지정하여, AI의 답변을 특정 JSON 구조로 강제할 수 있습니다. 
이를 통해 비정형적인 텍스트가 아닌, 예측 가능하고 바로 파싱하여 사용할 수 있는 구조화된 데이터를 얻을 수 있습니다.
참고: 이 기능은 `client.beta.chat.completions.parse`와 같이 `beta` 라이브러리를 사용합니다.

In [8]:
class MovieRecommendation(BaseModel):
    """영화 추천 정보의 구조를 정의합니다."""
    title: str = Field(description="영화 제목")
    genre: str = Field(description="장르")
    year: int = Field(description="개봉 연도")
    reason: str = Field(description="이 영화를 추천하는 이유")

def get_movie_recommendation(user_preference: str):
    """Structured Output을 사용하여 영화를 추천받는 함수입니다."""
    
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "user", "content": f"다음 취향에 맞는 영화 추천: {user_preference}"}
        ],
        response_format=MovieRecommendation  # AI가 이 Pydantic 모델 형식에 맞춰 응답합니다.
    )
    
    # .parse() 메서드는 응답을 자동으로 Pydantic 객체로 변환해줍니다.
    return response.choices[0].message.parsed

print("예제 4: Structured Output")
movie = get_movie_recommendation("우주를 배경으로 한 화려한 액션 SF 영화를 좋아해")

print(f"제목: {movie.title}")
print(f"장르: {movie.genre}")
print(f"연도: {movie.year}")
print(f"이유: {movie.reason}")

예제 4: Structured Output
제목: 가디언즈 오브 갤럭시
장르: 액션, SF
연도: 2014
이유: 우주를 배경으로 한 화려한 액션과 유머가 가득한 영화로, 다양한 캐릭터들이 펼치는 재미있는 스토리가 매력적입니다.


## 예제 5: 간단한 Agent - 여러 도구를 자동으로 사용

AI가 여러 도구(함수) 중에서 상황에 맞는 것을 스스로 선택하고, 필요한 경우 여러 번에 걸쳐 함수를 호출하도록 만들 수 있습니다.
사용자의 복합적인 요청("서울 날씨 알려주고, 15 곱하기 7도 계산해줘")을 해결하기 위해, AI는 `get_weather`와 `calculate` 함수를 순차적으로 호출합니다.

In [10]:
class CalculatorInput(BaseModel):
    """계산기 함수의 입력을 정의합니다."""
    expression: str = Field(description="계산할 수식 (예: '10 + 20 * 3')")

def calculate(expression: str):
    """주어진 수식을 계산하여 결과를 반환합니다."""
    try:
        # 주의: 실제 프로덕션 환경에서는 eval() 사용에 보안상 주의가 필요합니다.
        result = eval(expression)
        return {"result": result}
    except Exception as e:
        return {"error": f"계산 오류: {str(e)}"}

# AI에게 제공할 도구 목록을 정의합니다. (날씨, 계산기)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "특정 도시의 현재 날씨를 조회합니다",
            "parameters": WeatherInput.model_json_schema()
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "주어진 수식을 계산합니다",
            "parameters": CalculatorInput.model_json_schema()
        }
    }
]

# 함수 이름과 실제 함수 객체를 매핑합니다.
functions = {
    "get_weather": get_weather,
    "calculate": calculate
}

def simple_agent(user_query: str):
    """간단한 Agent 로직을 실행합니다."""
    messages = [{"role": "user", "content": user_query}]
    
    # AI가 여러 도구를 연쇄적으로 호출할 수 있도록 반복문을 사용합니다.
    for i in range(3):  # 최대 3번의 도구 사용을 허용합니다.
        print(f"\n[Agent Loop {i+1}] AI에게 요청...")
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
        
        message = response.choices[0].message
        messages.append(message) # 대화 기록에 AI의 응답을 추가합니다.
        
        # 만약 AI가 함수를 호출하지 않았다면, 답변을 반환하고 종료합니다.
        if not message.tool_calls:
            print("AI가 최종 답변을 생성했습니다.")
            return message.content
        
        # AI가 요청한 모든 함수를 실행합니다.
        print(f"🔧 AI가 {len(message.tool_calls)}개의 함수 호출을 요청했습니다.")
        for tool_call in message.tool_calls:
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            
            print(f"   - 실행할 함수: {func_name}({args})")
            
            # 실제 함수를 실행합니다.
            result = functions[func_name](**args)
            
            # 함수 실행 결과를 대화 기록에 추가합니다.
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps(result, ensure_ascii=False)
            })
            
    print("최대 반복 횟수에 도달했습니다.")
    return "작업을 완료했지만 최종 답변을 생성하지 못했습니다."

print("예제 5: 간단한 Agent")
query = "서울 날씨 알려주고, 15 곱하기 7도 계산해줘"
print(f"\n 질문: {query}")
response = simple_agent(query)
print(f"\n최종 답변: {response}")

예제 5: 간단한 Agent

💬 질문: 서울 날씨 알려주고, 15 곱하기 7도 계산해줘

[Agent Loop 1] AI에게 요청...
🔧 AI가 2개의 함수 호출을 요청했습니다.
   - 실행할 함수: get_weather({'city': '서울'})
   - 실행할 함수: calculate({'expression': '15 * 7'})

[Agent Loop 2] AI에게 요청...
AI가 최종 답변을 생성했습니다.

최종 답변: 서울의 현재 날씨는 22도 섭씨이며, 날씨 상태는 흐림입니다. 

또한, 15 곱하기 7의 계산 결과는 105입니다.


## 예제 6: 리스트 출력 - 여러 항목 반환

Structured Output 기능을 응용하여, 단일 객체가 아닌 여러 객체의 리스트를 반환하도록 요청할 수 있습니다.
예를 들어, 프로젝트에 대한 설명을 주면 여러 개의 '할 일(Task)' 객체로 구성된 리스트를 생성하도록 할 수 있습니다.

In [11]:
class Task(BaseModel):
    """개별 할 일 항목을 정의합니다."""
    title: str = Field(description="할 일의 제목")
    priority: str = Field(description="중요도 (high, medium, low 중 하나)")

class TaskList(BaseModel):
    """Task 객체들의 리스트를 정의합니다."""
    tasks: List[Task]

def get_task_list(description: str):
    """주어진 설명으로부터 할 일 목록을 생성합니다."""
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "user", "content": f"다음 상황에 대한 할 일 목록을 구체적으로 작성해줘: {description}"}
        ],
        response_format=TaskList # 응답 형식으로 TaskList를 지정합니다.
    )
    
    return response.choices[0].message.parsed

print("예제 6: 리스트 형식으로 응답받기")
task_list_obj = get_task_list("새로운 쇼핑몰 웹사이트 개발 프로젝트 시작")

for task in task_list_obj.tasks:
    print(f"- [우선순위: {task.priority}] {task.title}")

예제 6: 리스트 형식으로 응답받기
- [우선순위: high] 프로젝트 계획 수립
- [우선순위: high] 요구 사항 분석 회의 진행
- [우선순위: medium] 웹사이트 디자인 컨셉 정리
- [우선순위: high] 프로젝트 일정 및 마일스톤 설정
- [우선순위: high] 개발팀 구성 및 역할 분담
- [우선순위: medium] 필요한 기술 스택 선정
- [우선순위: high] 서버 및 도메인 준비
- [우선순위: medium] 프로토타입 제작 및 초기 사용자 피드백 수집
- [우선순위: high] 개발 환경 설정 및 개발 시작
- [우선순위: medium] QA 및 버그 수정 계획 세우기
- [우선순위: medium] 런칭 일정 및 마케팅 전략 수립
- [우선순위: high] 최종 점검 및 웹사이트 런칭
